## __TRABAJO DE INVESTIGACIÓN__ 
***
### __Métodos de Optimización 2022-2__
#### __Profesor__: Cristián Sepulveda S.
#### __Ayudante__: Tomás Lopez A.
#### __Integrantes__:  Estefanía Álvarez M. - Bastián Loyola J. - Carlos Retamales A.

***

#Se definen los datos del problema
n = cantidad de puntos limpios
m = cantidad de camiones
W = capacidad del camión m
w = peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio
#Columnas m: Costo de trasporte de cada camión desde cada punto limpio n
c = matrix(nrow = n, ncol = m)   


In [110]:
#Caso1: Factible
n = 5
m = 2
W = [100]
w = [10, 10, 10, 30, 50]
c = [0;1;2;3;4]


5-element Vector{Int64}:
 0
 1
 2
 3
 4

Con camiones

In [114]:
using JuMP, GLPK

# Problema del viajero, en este caso se busca el camino optimo para recoger desde un centro de reciclaje
# reciclaje de n puntos limpios, con m camiones, cada camión tiene una capacidad W
# y cada reciclaje tiene un peso w[i], el costo de transporte de cada reciclaje desde
# cada punto limpio es c[i,j]. 

#Consideraciones

# El camion solo puede pasar por un punto limpio una vez, excepto el punto de origen que puede pasar
# cuantas veces sea necesario
# Si el camion pasa por el punto de origen su capacidad se restablece a W
# Se desea saber la ruta de cada camion y el costo total de transporte

#Se define el modelo
model = Model(GLPK.Optimizer)

#Se definen los datos del problema
n = 5 #cantidad de puntos limpios
m = 2 #cantidad de camiones
W = [100,120] # capacidad del camión m
w = [0, 10, 10, 30, 50] #peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio
c = [0 0 ;1 1;2 2;3 3;4 4]

#Se definen las variables
#binarias
#camión j que recoge el reciclaje del punto limpio i
@variable(model, x[1:n, 1:m], Bin)


#Función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:m))

#Restricciones

#1.Un camion puede recoger uno o mas residuos
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#2.Un residuo solo puede ser recogido por uno o mas camiones
@constraint(model, [i in 1:n], sum(x[i,j] for j in 1:m) >= 1)

#3.La capacidad del camion no puede ser sobrepasada
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j])

#4.El camion puede solo pasar una vez por cada punto limpio, excepto el punto de origen
@constraint(model, [i in 2:n, j in 1:m], x[i,j] <= sum(x[k,j] for k in 1:n if k != i))

#5.El camion puede pasar por el punto de origen cuantas veces sea necesario
@constraint(model, [j in 1:m], sum(x[1,j] for i in 1:n) >= 1)

#6.El camion si pasa por el punto de origen su capacidad se restablece a W
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j] + W[j]*x[1,j])

#Se resuelve el modelo
optimize!(model)

#Se imprime el resultado
println("Valor óptimo: ", objective_value(model))

#Se imprime la ruta de cada camión
for j in 1:m
    println("Camión ", j)
    for i in 1:n
        if value(x[i,j]) == 1
            println("Punto ", i)
        end
    end
end


Valor óptimo: 10.0
Camión 1
Punto 1
Punto 3
Punto 4
Punto 5
Camión 2
Punto 1
Punto 2


In [ ]:
using JuMP, GLPK

# Problema del viajero, en este caso se busca el camino optimo para recoger desde un centro de reciclaje
# reciclaje de n puntos limpios, con m camiones, cada camión tiene una capacidad W
# y cada reciclaje tiene un peso w[i], el costo de transporte de cada reciclaje desde
# cada punto limpio es c[i,j]. 

#Consideraciones

# El camion solo puede pasar por un punto limpio una vez, excepto el punto de origen que puede pasar
# cuantas veces sea necesario
# Si el camion pasa por el punto de origen su capacidad se restablece a W
# Se desea saber la ruta de cada camion y el costo total de transporte

#Se define el modelo
model = Model(GLPK.Optimizer)

#Se definen los datos del problema
n = 5 #cantidad de puntos limpios
m = 1 #cantidad de camiones
W = [100] # capacidad del camión m
w = [0, 10, 10, 30, 50] #peso de cada reciclaje
#costo de transporte de cada residuo desde cada punto limpio
c = [0;1;2;3;4]

#Se definen las variables
#binarias
#camión j que recoge el reciclaje del punto limpio i
@variable(model, x[1:n, 1:m], Bin)


#Función objetivo
@objective(model, Min, sum(c[i,j]*x[i,j] for i in 1:n, j in 1:m))

#Restricciones

#1.Un camion puede recoger uno o mas residuos
@constraint(model, [j in 1:m], sum(x[i,j] for i in 1:n) >= 1)

#2.Un residuo solo puede ser recogido por un camion
@constraint(model, [i in 1:n], sum(x[i,j] for j in 1:m) == 1)

#3.La capacidad del camion no puede ser sobrepasada
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j])

#4.El camion puede solo pasar una vez por cada punto limpio, excepto el punto de origen
@constraint(model, [i in 2:n, j in 1:m], x[i,j] <= sum(x[k,j] for k in 1:n if k != i))

#5.El camion puede pasar por el punto de origen cuantas veces sea necesario
@constraint(model, [j in 1:m], sum(x[1,j] for i in 1:n) >= 1)

#6.El camion si pasa por el punto de origen su capacidad se restablece a W
@constraint(model, [j in 1:m], sum(w[i]*x[i,j] for i in 1:n) <= W[j] + W[j]*x[1,j])








#Se resuelve el modelo
optimize!(model)

#Se imprime el resultado
println("Valor óptimo: ", objective_value(model))

#Se imprime la ruta de cada camión
for j in 1:m
    println("Camión ", j)
    for i in 1:n
        if value(x[i,j]) == 1
            println("Punto ", i)
        end
    end
end


Con Camiones y Drones

Solución óptima: 0.0
